<a href="https://colab.research.google.com/github/webbigdata-jp/python_sample/blob/main/gemma_2_2b_jpn_it_translate_gguf_Free_Colab_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

gemma-2-2b-jpn-it-translate-ggufは日英・英日に特化した機械翻訳モデル[webbigdata/gemma-2-2b-jpn-it-translate-gguf](https://huggingface.co/webbigdata/gemma-2-2b-jpn-it-translate-gguf)をgpuがなくても動くように変換したサンプルです。  このColabのサンプルに従うと、GPUがないMacやColabでも動かす事ができます。    

gemma-2-2b-jpn-it-translate-gguf is a sample that converts a machine translation model[webbigdata/gemma-2-2b-jpn-it-translate-gguf](https://huggingface.co/webbigdata/gemma-2-2b-jpn-it-translate-gguf) specialized for Japanese-English and English-Japanese to run without a gpu.  If you follow this Colab sample, you can run it on a Mac or Colab without a GPU.


llama.cppを使うと、様々な量子化手法でファイルのサイズを小さくする事が出来ます。本サンプルでは4種類のみを扱います。小さいサイズのモデルは、少ないメモリで高速に動作させることができますが、モデルの性能も低下します。4ビット(q4_k_m)くらいがバランスが良いと言われていますが、本サンプルコードでは特定の文章を全モデルで翻訳し、どのモデルが貴方の作業に適切かを確認できるようにしたものです。gguf版はパラメーターの調整をしないと幻覚が増えます。  

Although llama.cpp can be used to reduce the size of the file with various quantization methods, this sample deals with only 4 types. Smaller models can run faster with less memory, but the performance of the models is also reduced. 4 bits (q4_k_m) is said to be a good balance, but this sample code translates a particular sentence with all models so that you can see which model is appropriate for your work. This sample code translates a specific sentence across all models so that you can see which model is appropriate for your work.The GGUF version will have more hallucinations if you don't adjust the parameters.  

- gemma-2-2b-jpn-it-translate-IQ3_XXS.gguf 1.18GB
- gemma-2-2b-jpn-it-translate-Q4_K_L.gguf 1.85GB


必要に応じて以下のパラメーターを調整してください  

- 温度（--temp）: この値を下げると、モデルがより確信度の高い（つまり、より一般的な）単語を選択する傾向が強くなります。
- トップP（--top_p）: この値をさらに低く設定することで、モデルが考慮する単語の範囲を狭め、より一貫性のあるテキストを生成するようになります。
- 生成する単語数（-n）: この値を減らすことで、モデルが生成するテキストの長さを短くし、不要な追加テキストの生成を防ぐことができます。-1 = 無限大、-2 = 文脈が満たされるまで。

以下はllama.cppの作者(ggerganov)による推奨パラメーターです

- -e (改行\nをエスケープ)
- --temp 0 (最も確率の高いトークンのみを選択)
- --repeat-penalty 1.0 (繰り返しペナルティをオフ)

Adjust the following parameters as needed  
- Temperature (--temp): Lowering this value will make the model more likely to select more confident (i.e., more common) words.
- Top P (--top_p): Setting this value even lower will narrow the range of words considered by the model and produce more consistent text.
- Number of words to generate (-n): Reducing this value will shorten the length of text generated by the model and prevent the generation of unnecessary additional text. -1 = infinity(default), -2 = until context filled.

The following are the recommended parameters by the author of llama.cpp(ggerganov)
- -e (escape newlines (\n))
- --temp 0(pick most probable tokens)
- --repeat-penalty 1.0(disable repetition penalty (it's never a good idea to have this with instruction tuned models)


上段メニューより「ランタイム」→「全てのセルを実行」で実行できます。  
You can run this by selecting "Runtime" > "Run All Cells" from the top menu.  

改行を含む複数の文章を一括して翻訳する事はできません。文章は一文ずつ、チャット形式で入力してください。以下の例ではプロンプトテンプレートを手動で設定していますが、[gemma-2-2b-jpn-it-translate-gguf page](https://huggingface.co/webbigdata/gemma-2-2b-jpn-it-translate-gguf)のサンプルなどを参考にapply_chat_templateを使って変換する事をお勧めします。  




It is not possible to translate multiple sentences including line breaks at once. Please enter each sentence in chat format. In the following example, the prompt template is set manually, but we recommend using apply_chat_template to convert the text, as shown in the sample on [gemma-2-2b-jpn-it-translate-gguf page](https://huggingface.co/webbigdata/gemma-2-2b-jpn-it-translate-gguf).




In [5]:
import os
import subprocess

os.environ['TRANSLATE_TEMPLATE_Ja'] = """<start_of_turn>user
You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the original language or use appropriate translations as necessary. Take a deep breath, calm down, and start translating.

Translate Japanese to English.
When translating, please use the following hints:
[writeing_style: web-fiction]
[羂索: Kenjyaku]
<start_of_turn>model
OK<end_of_turn>
<start_of_turn>user
羂索 （美しい、本気の土下座だ…私がこの域に達したのは20代後半……)<end_of_turn>
<start_of_turn>model
"""

os.environ['TRANSLATE_TEMPLATE_En'] = """<start_of_turn>user
You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in the original language or use appropriate translations as necessary. Take a deep breath, calm down, and start translating.

Translate English to Japanese.
When translating, please use the following hints:
[writing_style: casual, nsfw]
[Geoffrey Hinton: ジェフェリー・ヒントン]
[Sam Altman: サム・アルトマン]
[Ilya Sutskever: イリヤ・サツケバー]
<start_of_turn>model
OK<end_of_turn>
<start_of_turn>user
Geoffrey Hinton: I am particularly proud of the fact that one of my students (Ilya Sutskever) fired Sam Altman.<end_of_turn>
<start_of_turn>model
"""

os.environ['BASE_URL'] = "https://huggingface.co/webbigdata/gemma-2-2b-jpn-it-translate-gguf/resolve/main/gemma-2-2b-jpn-it-translate"

In [6]:
%%bash
# setup llama.cpp
git clone https://github.com/ggerganov/llama.cpp.git
cd llama.cpp
make -j

I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE  -std=c11   -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -fopenmp -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE 
I NVCCFLAGS: -std=c++11 -O3 -g 
I LDFLAGS:    
I CC:     

fatal: destination path 'llama.cpp' already exists and is not an empty directory.


## gemma-2-2b-jpn-it-translate-IQ3_XXS.gguf

In [16]:
%%bash
cd llama.cpp
wget ${BASE_URL}-IQ3_XXS.gguf 2>>IQ3_XXS.gguf.log

In [17]:
%%bash
cd llama.cpp && ./llama-cli -e --temp 0 --repeat-penalty 1.0 -n -2 -m 'gemma-2-2b-jpn-it-translate-IQ3_XXS.gguf' -p "$TRANSLATE_TEMPLATE_Ja" 2>Q3_K_M.log

user
You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in th

In [18]:
%%bash
cd llama.cpp && ./llama-cli -e --temp 0 --repeat-penalty 1.0 -n -2 -m 'gemma-2-2b-jpn-it-translate-IQ3_XXS.gguf' -p "$TRANSLATE_TEMPLATE_En" 2>>Q3_K_M.log

user
You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in th

## gemma-2-2b-jpn-it-translate-Q4_K_L.gguf

In [19]:
%%bash
cd llama.cpp
wget ${BASE_URL}-Q4_K_L.gguf 2>>q4_k.log

In [20]:
%%bash
cd llama.cpp && ./llama-cli -e --temp 0 --repeat-penalty 1.0 -n -2 -m 'gemma-2-2b-jpn-it-translate-Q4_K_L.gguf' -p "$TRANSLATE_TEMPLATE_Ja" 2>Q4_K.log

user
You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in th

In [21]:
%%bash
cd llama.cpp && ./llama-cli -e --temp 0 --repeat-penalty 1.0 -n -2 -m 'gemma-2-2b-jpn-it-translate-Q4_K_L.gguf' -p "$TRANSLATE_TEMPLATE_En" 2>>Q4_K.log

user
You are a highly skilled professional Japanese-English and English-Japanese translator. Translate the given text accurately, taking into account the context and specific instructions provided. Steps may include hints enclosed in square brackets [] with the key and value separated by a colon:. Only when the subject is specified in the Japanese sentence, the subject will be added when translating into English. If no additional instructions or context are provided, use your expertise to consider what the most appropriate context is and provide a natural translation that aligns with that context. When translating, strive to faithfully reflect the meaning and tone of the original text, pay attention to cultural nuances and differences in language usage, and ensure that the translation is grammatically correct and easy to read. After completing the translation, review it once more to check for errors or unnatural expressions. For technical terms and proper nouns, either leave them in th

## 謝辞 Acknowledgment

Original Base Model  
[google/gemma-2-2b-jpn-it](https://huggingface.co/google/gemma-2-2b-jpn-it)  

This gguf model  
[webbigdata/gemma-2-2b-jpn-it-translate-gguf](https://huggingface.co/webbigdata/gemma-2-2b-jpn-it-translate-gguf)  

This work was done by : [webbigdata](https://webbigdata.jp/)